# uCB-seq HEK and PreAD Imaging Final

___

## Load Packages:

In [1]:
suppressWarnings(suppressMessages({
    library(rbin)
    library(data.table)
    library(scater)
    library(tidyverse)
    library(annotables)
    library(SummarizedExperiment)
    library(RColorBrewer)
    library(SingleCellExperiment)
    library(DescTools)
    library(gplots)
    library(eulerr)
    library(grid)
    library(gridExtra) 
    library(GenomicFeatures)
    library(GenomicRanges)
    library(BSgenome)
    }))

## Temporary Functions

In [2]:
setwd('C:/Users/tyler/Desktop/Downsampled zUMIs/out/uCBSeqHEK/zUMIs_output/expression')
uCBSeqHEK <- readRDS('uCBSeqHEK.dgecounts.rds')

setwd('C:/Users/tyler/Desktop/Downsampled zUMIs/out/uCBSeqHEKPreAD/zUMIs_output/expression')
uCBSeqHEKPreAD <- readRDS('uCBSeqHEKPreAD.dgecounts.rds')

setwd('C:/Users/tyler/Desktop/Downsampled zUMIs/out/uCBSeqPreAD/zUMIs_output/expression')
uCBSeqPreAD <- readRDS('uCBSeqPreAD.dgecounts.rds')

## Preanalysis Processing

In [3]:
# Remove duplicate genes in dedup_grch38.
toRemove <- vector()
last <- 1
for (i in 2:(nrow(grch38))) {
    if (grch38[i,1] == grch38[i - 1,1]) {
        toRemove[last] <- i
        last <- last + 1
    }
}

# Create deuplicated grch38
dedup_grch38 <- grch38[-toRemove,]

# Check for duplicates in dedup_grch38
postDupCount <- 0
for (i in 1:(nrow(dedup_grch38) - 1)) {
    if (dedup_grch38[i,1] == dedup_grch38[i + 1,1]) {
        postDupCount <- postDupCount + 1
    }
}
print(paste0("There are ", postDupCount, " duplicate genes in dedup_grch38!"))

[1] "There are 0 duplicate genes in dedup_grch38!"


___

# Set Parameters for Analysis

In [4]:
# Initialize samples, barcodes, and sampleslist
experiment <- 'All Comparisons'
samples <- c('uCBSeqHEKPreAD', 'uCBSeqHEK', 'uCBSeqPreAD')
barcodesHEKPreAD <- c('TCACAGCA', 'GTAGCACT', 'ATAGCGTC', 'TGCTACAG','CGCTATGA', 'CATCGTGA', 'GGCATTGT')
barcodesHEK <- c('TCACAGCA', 'GTAGCACT', 'TGCTACAG','CGCTATGA', 'ATGCACGT', 'TATGCACG', 'CATCGTGA', 'GGCATTGT')
barcodesPreAD <- c('TCACAGCA', 'GTAGCACT', 'CGCTATGA', 'ATGCACGT', 'TATGCACG', 'CATCGTGA', 'GGCATTGT') 
HEKPreADSamplesList <- c('HEKPreAD_TCACAGCA', 'HEKPreAD_GTAGCACT', 'HEKPreAD_ATAGCGTC', 'HEKPreAD_TGCTACAG','HEKPreAD_CGCTATGA', 'HEKPreAD_CATCGTGA', 'HEKPreAD_GGCATTGT')
HEKSamplesList <- c('HEK_TCACAGCA', 'HEK_GTAGCACT', 'HEK_TGCTACAG','HEK_CGCTATGA', 'HEK_ATGCACGT', 'HEK_TATGCACG', 'HEK_CATCGTGA', 'HEK_GGCATTGT')
PreADSamplesList <- c('PreAD_TCACAGCA', 'PreAD_GTAGCACT', 'PreAD_CGCTATGA', 'PreAD_ATGCACGT', 'PreAD_TATGCACG', 'PreAD_CATCGTGA', 'PreAD_GGCATTGT') 

# Initialize Samples List
samplesList <- NULL
i <- 1
for (sample in samples) {
    if (sample == "uCBSeqHEKPreAD") {
        for (barcode in barcodesHEKPreAD) {
            eval(parse(text = paste0('samplesList[i] <- ', "'", sample, "_", barcode, "'")))
            i  <- i + 1
        }
    } else if (sample == "uCBSeqHEK") {
        for (barcode in barcodesHEK) {
            eval(parse(text = paste0('samplesList[i] <- ', "'", sample, "_", barcode, "'")))
            i  <- i + 1
        }  
    } else if (sample == "uCBSeqPreAD") {
            for (barcode in barcodesPreAD) {
            eval(parse(text = paste0('samplesList[i] <- ', "'", sample, "_", barcode, "'")))
            i  <- i + 1
        }  
    }

}

# Initialize Negatives List
fullNegList <- c()

In [5]:
# Looking at exon, intron, or intron and exon (intron.exon)
lookAt <- 'inex'

In [6]:
# Downsampling value to use for primary analysis
downsamplingSC <- '125000'

# Input Experimental Data

In [7]:
# Initialize colData
colDataNames <- c('Sample_ID', 'Cell_ID', 'NCells', 'Format', 'TotalReads', 'AlignedReads', 'AlignedFrac', 'AlignedReadsExon', 
                    'ExonFrac', 'AlignedReadsIntron', 'IntronFrac', 'AlignedReadsIntergenic', 'IntergenicFrac', 'GenesDetected', 
                    'UMIsDetected')

# Initialize Cell ID, input, etc. 
Cell_ID <- rep('HEKPreAD', length(samplesList))
NCells <- c(rep('Single Cell', length(samplesList)))
Format <- c(rep('uCB-seq', length(samplesList)))


# Initialize colData to fill in
TotalReads  <- double(length = length(samplesList))
AlignedReads <- double(length = length(samplesList))
AlignedFrac <- double(length = length(samplesList))
AlignedReadsExon <- double(length = length(samplesList))
ExonFrac <- double(length = length(samplesList))
AlignedReadsIntron <- double(length = length(samplesList))
IntronFrac <- double(length = length(samplesList))
AlignedReadsIntergenic <- double(length = length(samplesList))
IntergenicFrac <- double(length = length(samplesList))
AlignedReadsAmbiguous <- double(length = length(samplesList))
AmbiguousFrac <- double(length = length(samplesList))
GenesDetected <- double(length = length(samplesList))
UMIsDetected  <- double(length = length(samplesList))

# Initialize metaData
metaDataNames <- c('Experiment', 'Sample_ID')
Experiment <- c(rep('uCBSeqHEKPreAD', length(barcodesHEKPreAD)), rep('uCBSeqHEK', length(barcodesHEK)), rep('uCBSeqPreAD', length(barcodesPreAD)))
Sample_ID <- samplesList

___

## Build Expression Matrix from Sequencing Dataset

### Creating Individual Expression Matrices for All Samples as "______Counts"

In [8]:
# Create all individual expression matrices

i = 0
for (sample in samples) {
    if (sample == "uCBSeqHEKPreAD") {
        setwd(paste0("C:/Users/tyler/Desktop/Downsampled zUMIs/out/uCBSeqHEKPreAD/zUMIs_output/expression"))
        matrixName <- paste0(sample, '_', 'Counts')
        fileName <- paste0(sample, '.dgecounts.rds')
        eval(parse(text = paste0(matrixName, " <- readRDS(","'", fileName, "'", ")")))
        for (barcode in barcodesHEKPreAD) {
            umiCountsName <- paste0(sample, '_', barcode, '_', 'UMICounts')
            umiFieldsName <- paste0(matrixName, "$umicount$", lookAt, "$downsampling$downsampled_", downsamplingSC, "[,", "'", barcode, "']")
            eval(parse(text = paste0(umiCountsName, " <- as.matrix(", umiFieldsName, ")")))

            readCountsName <- paste0(sample, '_', barcode, '_', 'ReadCounts')
            readFieldsName <- paste0(matrixName, "$readcount$", lookAt, "$downsampling$downsampled_", downsamplingSC, "[,", "'", barcode, "']")
            eval(parse(text = paste0(readCountsName, " <- as.matrix(", readFieldsName, ")")))
        }
    } else if (sample == "uCBSeqHEK") {
        setwd(paste0("C:/Users/tyler/Desktop/Downsampled zUMIs/out/uCBSeqHEK/zUMIs_output/expression"))
        matrixName <- paste0(sample, '_', 'Counts')
        fileName <- paste0(sample, '.dgecounts.rds')
        eval(parse(text = paste0(matrixName, " <- readRDS(","'", fileName, "'", ")")))
        for (barcode in barcodesHEK) {
            if (barcode == "CGCTATGA") {
                umiCountsName <- paste0(sample, '_', barcode, '_', 'UMICounts')
                umiFieldsName <- paste0(matrixName, "$umicount$", lookAt, "$all[,", "'", barcode, "']")
                eval(parse(text = paste0(umiCountsName, " <- as.matrix(", umiFieldsName, ")")))

                readCountsName <- paste0(sample, '_', barcode, '_', 'ReadCounts')
                readFieldsName <- paste0(matrixName, "$readcount$", lookAt, "$all[,", "'", barcode, "']")
                eval(parse(text = paste0(readCountsName, " <- as.matrix(", readFieldsName, ")")))
            } else {
                umiCountsName <- paste0(sample, '_', barcode, '_', 'UMICounts')
                umiFieldsName <- paste0(matrixName, "$umicount$", lookAt, "$downsampling$downsampled_", downsamplingSC, "[,", "'", barcode, "']")
                eval(parse(text = paste0(umiCountsName, " <- as.matrix(", umiFieldsName, ")")))

                readCountsName <- paste0(sample, '_', barcode, '_', 'ReadCounts')
                readFieldsName <- paste0(matrixName, "$readcount$", lookAt, "$downsampling$downsampled_", downsamplingSC, "[,", "'", barcode, "']")
                eval(parse(text = paste0(readCountsName, " <- as.matrix(", readFieldsName, ")")))
            }
        }
    } else if (sample == "uCBSeqPreAD") {
        setwd(paste0("C:/Users/tyler/Desktop/Downsampled zUMIs/out/uCBSeqPreAD/zUMIs_output/expression"))
        matrixName <- paste0(sample, '_', 'Counts')
        fileName <- paste0(sample, '.dgecounts.rds')
        eval(parse(text = paste0(matrixName, " <- readRDS(","'", fileName, "'", ")")))
        for (barcode in barcodesPreAD) {
            if (barcode == "ATGCACGT") {
                umiCountsName <- paste0(sample, '_', barcode, '_', 'UMICounts')
                umiFieldsName <- paste0(matrixName, "$umicount$", lookAt, "$all[,", "'", barcode, "']")
                eval(parse(text = paste0(umiCountsName, " <- as.matrix(", umiFieldsName, ")")))

                readCountsName <- paste0(sample, '_', barcode, '_', 'ReadCounts')
                readFieldsName <- paste0(matrixName, "$readcount$", lookAt, "$all[,", "'", barcode, "']")
                eval(parse(text = paste0(readCountsName, " <- as.matrix(", readFieldsName, ")")))
            } else {
                umiCountsName <- paste0(sample, '_', barcode, '_', 'UMICounts')
                umiFieldsName <- paste0(matrixName, "$umicount$", lookAt, "$downsampling$downsampled_", downsamplingSC, "[,", "'", barcode, "']")
                eval(parse(text = paste0(umiCountsName, " <- as.matrix(", umiFieldsName, ")")))

                readCountsName <- paste0(sample, '_', barcode, '_', 'ReadCounts')
                readFieldsName <- paste0(matrixName, "$readcount$", lookAt, "$downsampling$downsampled_", downsamplingSC, "[,", "'", barcode, "']")
                eval(parse(text = paste0(readCountsName, " <- as.matrix(", readFieldsName, ")")))
            }
        }
    }
}

### Building Master UMI Expression Matrix and Master Readcount Expression Matrix

In [9]:
# Initialize namesList with first sample
eval(parse(text = paste0("namesList <- names(", samplesList[1], "_UMICounts[,1])")))

# Iteratively add genes present in samples but missing from namesList until namesList is fully populated
for (sample in samples) {
    if (sample == "uCBSeqHEKPreAD") {
        for (barcode in barcodesHEKPreAD) {
            if (sample == samples[1] & barcode == barcodesHEKPreAD[1]) {
            } else {
                eval(parse(text = paste0("namesList <- c(namesList, names(", sample, "_", barcode, 
                                         "_UMICounts[,1])[!names(", sample, "_", barcode, 
                                         "_UMICounts[,1]) %in% namesList])")))
            }
        }
    } else if (sample == "uCBSeqHEK") {
        for (barcode in barcodesHEK) {
            if (sample == samples[1] & barcode == barcodesHEK[1]) {
            } else {
                eval(parse(text = paste0("namesList <- c(namesList, names(", sample, "_", barcode, 
                                         "_UMICounts[,1])[!names(", sample, "_", barcode, 
                                         "_UMICounts[,1]) %in% namesList])")))
            }
        }
    } else if (sample == "uCBSeqPreAD") {
        for (barcode in barcodesPreAD) {
            if (sample == samples[1] & barcode == barcodesPreAD[1]) {
            } else {
                eval(parse(text = paste0("namesList <- c(namesList, names(", sample, "_", barcode, 
                                         "_UMICounts[,1])[!names(", sample, "_", barcode, 
                                         "_UMICounts[,1]) %in% namesList])")))
            }
        }
    }
}

# Do the same for bulk HEK sample
setwd("C:/Users/tyler/Desktop/Downsampled zUMIs/out/HEK_Bulk/")
temp <- as.matrix(read.csv('HEK_Bulk.csv'))
bulkMatrix <- as.numeric(as.matrix(temp[,3]))
# Establish ENSG ID as rownames
names(bulkMatrix) <- temp[,1]
# Add bulk genes to namesList
namesList <- c(namesList, names(bulkMatrix)[!names(bulkMatrix) %in% namesList])

# Sort namesList
namesList <- sort(namesList)

# Create matrix with all zeros containing all names for comparison against all samples
namesMatrix <- as.vector(numeric(length(namesList)), "list")
names(namesMatrix) = namesList

In [10]:
#Define function fillSparseMatrix to fill out and order each sample's matrix

fillSparseMatrix <- function(inputMatrix, namesList) {
    missingNames <- names(namesMatrix)[!names(namesMatrix) %in% names(inputMatrix[,1])]
    missingNames
    missingMatrix <- as.vector(numeric(length(missingNames)), "list")
    names(missingMatrix) <- missingNames
    fullInputMatrix <- do.call(cbind, c(inputMatrix[,1], missingMatrix))
    fullInputMatrix <- fullInputMatrix[, order(names(fullInputMatrix[1,]))]
    return(fullInputMatrix)
}

In [11]:
# Run fillSparseMatrix on all UMI samples
for (sampleName in samplesList) {
    eval(parse(text = paste0(sampleName, " <- fillSparseMatrix(", sampleName, "_UMICounts)")))
}

# Run fillSparseMatrix on Bulk
    missingNames <- names(namesMatrix)[!names(namesMatrix) %in% names(bulkMatrix)]
    missingMatrix <- as.vector(numeric(length(missingNames)), "list")
    names(missingMatrix) <- missingNames
    fullInputMatrix <- do.call(cbind, c(bulkMatrix, missingMatrix))
    Bulk <- fullInputMatrix[, order(names(fullInputMatrix[1,]))]

# Bind all filled matrices into final umiMatrix
eval(parse(text = paste0("temp <- rbind(", samplesList[1], ", ", samplesList[2], ")")))

for (i in 3:length(samplesList)) {
    if (i == length(samplesList)) {
        eval(parse(text = paste0("umiMatrix <- rbind(temp, ", samplesList[i], ")")))
    } else {
        eval(parse(text = paste0("temp <- rbind(temp, ", samplesList[i], ")")))
    }
}

umiMatrix <- t(umiMatrix)

In [12]:
# Run fillSparseMatrix on all readcount samples
for (sampleName in samplesList) {
    eval(parse(text = paste0(sampleName, " <- fillSparseMatrix(", sampleName, "_ReadCounts)")))
}

# Bind all filled matrices into final umiMatrix
eval(parse(text = paste0("temp <- rbind(", samplesList[1], ", ", samplesList[2], ")")))

for (i in 3:length(samplesList)) {
    if (i == length(samplesList)) {
        eval(parse(text = paste0("readMatrix <- rbind(temp, ", samplesList[i], ")")))
    } else {
        eval(parse(text = paste0("temp <- rbind(temp, ", samplesList[i], ")")))
    }
}

readMatrix <- t(readMatrix)

_____

# Pull GC and Length Genomic Information

In [13]:
# Pull txdb info from Ensembl directly
# txdb <- makeTxDbFromEnsembl(organism="Homo sapiens",
#                     release=NA,
#                     circ_seqs=DEFAULT_CIRC_SEQS,
#                     server="ensembldb.ensembl.org")

In [14]:
# Pull txdb info from saved sqlite of Ensembl database
setwd('C:/Users/tyler/OneDrive/Streets Lab/Jupyter/')
txdb <- loadDb("ensemblTxdb.sqlite")

In [15]:
# Get transcripts by Gene, tidy for only Chr1-22, X, Y, MT 
tx_by_gene <- transcriptsBy(txdb, 'gene')
rm(txdb)
genome <- getBSgenome("GRCh38")
seqlevels(tx_by_gene, pruning.mode = "tidy") <- c('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y', 'MT')
genome(tx_by_gene) <- "GRCh38"

In [16]:
# Extract the transcript sequences grouped by gene:
txseq_by_gene <- getSeq(genome, tx_by_gene)
rm(tx_by_gene)
# Compute the transcript GC, AT, and total nucleotide counts grouped by gene:
af <- alphabetFrequency(unlist(txseq_by_gene, use.names=FALSE), baseOnly=TRUE)
txGCcount_by_gene <- relist(af[ , "C"] + af[ , "G"], txseq_by_gene)
txlength_by_gene <- relist(af[ , "C"] + af[ , "G"] + af[,"A"] + af[,"T"], txseq_by_gene)
txATcount_by_gene <- relist(af[,"A"] + af[,"T"], txseq_by_gene)
rm(af)

In [17]:
# Create dataframe with %GC and length by Ensembl ID
geneData <- as.data.frame(txGCcount_by_gene)
colnames(geneData)[colnames(geneData)=="value"] <- "GC_count"
colnames(geneData)[colnames(geneData)=="group_name"] <- "Ensembl ID"
geneData$Length = as.data.frame(txlength_by_gene)[,'value']
geneData$GC = geneData$GC_count / geneData$Length
geneData <- geneData[,-which(names(geneData) %in% c('group', 'GC_count'))]
geneData <- split(geneData, geneData$'Ensembl ID')

____

## Convert Matrices to Tibbles for Visualization

In [ ]:
# Create umiTibble and readTibble

# Number of genedata variables (e.g. Ensembl ID, Gene Symbol, etc.)
numGeneData <- 12

# Create and fill umiTibble and readTibble
umiTibble <- tibble('Ensembl ID' = names(umiMatrix[,1]))
readTibble <- tibble('Ensembl ID' = names(readMatrix[,1]))
for (ID in names(umiMatrix[,1])) {
    ti <- umiTibble[,1] == ID
    gi <- dedup_grch38[,1] == ID
    
    # Input basic GRCh38 Gene Data
    if (sum(gi) != 0) {
        umiTibble[ti, 'Gene Symbol'] <- dedup_grch38[gi, "symbol"]
        umiTibble[ti, 'Biotype'] <- dedup_grch38[gi, "biotype"]
        umiTibble[ti, 'Description'] <- dedup_grch38[gi, "description"]
        readTibble[ti, 'Gene Symbol'] <- dedup_grch38[gi, "symbol"]
        readTibble[ti, 'Biotype'] <- dedup_grch38[gi, "biotype"]
        readTibble[ti, 'Description'] <- dedup_grch38[gi, "description"]
    } else {
        umiTibble[ti, 'Gene Symbol'] <- NA
        umiTibble[ti, 'Biotype'] <- NA
        umiTibble[ti, 'Description'] <- NA
        readTibble[ti, 'Gene Symbol'] <- NA
        readTibble[ti, 'Biotype'] <- NA
        readTibble[ti, 'Description'] <- NA
    }
    
    # Input gene and transcript-level data
    if (eval(parse(text = paste0('is.null(geneData$', ID, ')')))) {
        umiTibble[ti, 'MaxLen'] <- NA
        umiTibble[ti, 'MinLen'] <- NA
        umiTibble[ti, 'MeanLen'] <- NA
        umiTibble[ti, 'MedianLen'] <- NA
        umiTibble[ti, 'MaxGC'] <- NA
        umiTibble[ti, 'MinGC'] <- NA
        umiTibble[ti, 'MeanGC'] <- NA
        umiTibble[ti, 'MedianGC'] <- NA
        
        readTibble[ti, 'MaxLen'] <- NA
        readTibble[ti, 'MinLen'] <- NA
        readTibble[ti, 'MeanLen'] <- NA
        readTibble[ti, 'MedianLen'] <- NA
        readTibble[ti, 'MaxGC'] <- NA
        readTibble[ti, 'MinGC'] <- NA
        readTibble[ti, 'MeanGC'] <- NA
        readTibble[ti, 'MedianGC'] <- NA
    } else {
        umiTibble[ti, 'MaxLen'] <- eval(parse(text = paste0('max(geneData$', ID, '[,"Length"])')))
        umiTibble[ti, 'MinLen'] <- eval(parse(text = paste0('min(geneData$', ID, '[,"Length"])')))
        umiTibble[ti, 'MeanLen'] <- eval(parse(text = paste0('mean(geneData$', ID, '[,"Length"])')))
        umiTibble[ti, 'MedianLen'] <- eval(parse(text = paste0('median(geneData$', ID, '[,"Length"])')))
        umiTibble[ti, 'MaxGC'] <- eval(parse(text = paste0('max(geneData$', ID, '[,"GC"])')))
        umiTibble[ti, 'MinGC'] <- eval(parse(text = paste0('min(geneData$', ID, '[,"GC"])')))
        umiTibble[ti, 'MeanGC'] <- eval(parse(text = paste0('mean(geneData$', ID, '[,"GC"])')))
        umiTibble[ti, 'MedianGC'] <- eval(parse(text = paste0('median(geneData$', ID, '[,"GC"])')))
        
        readTibble[ti, 'MaxLen'] <- eval(parse(text = paste0('max(geneData$', ID, '[,"Length"])')))
        readTibble[ti, 'MinLen'] <- eval(parse(text = paste0('min(geneData$', ID, '[,"Length"])')))
        readTibble[ti, 'MeanLen'] <- eval(parse(text = paste0('mean(geneData$', ID, '[,"Length"])')))
        readTibble[ti, 'MedianLen'] <- eval(parse(text = paste0('median(geneData$', ID, '[,"Length"])')))
        readTibble[ti, 'MaxGC'] <- eval(parse(text = paste0('max(geneData$', ID, '[,"GC"])')))
        readTibble[ti, 'MinGC'] <- eval(parse(text = paste0('min(geneData$', ID, '[,"GC"])')))
        readTibble[ti, 'MeanGC'] <- eval(parse(text = paste0('mean(geneData$', ID, '[,"GC"])')))
        readTibble[ti, 'MedianGC'] <- eval(parse(text = paste0('median(geneData$', ID, '[,"GC"])')))
    }
}

# Add samples into Tibble
for (i in 1:length(samplesList)) {
    eval(parse(text = paste0('umiTibble$', samplesList[i], " = umiMatrix[,i]")))
    eval(parse(text = paste0('readTibble$', samplesList[i], " = readMatrix[,i]")))    
}

In [ ]:
# Add Summary Data (including bulk data) into Tibble
numSummaryData <- 1

# Add Bulk HEK Counts into Tibble
umiTibble$Bulk = Bulk

In [ ]:
# View umiTibble 
umiTibble
readTibble

In [ ]:
# write.table(umiTibble, file = 'umiTibble_uCBSeqImaging.csv', append = FALSE, sep = ",", row.names = FALSE)

In [100]:
# Remove rRNA from Tibble
# mtRNALocationUMI <- which(umiTibble$Biotype == 'Mt_rRNA')
# umiTibble <- umiTibble[-mtRNALocationUMI,]

# mtRNALocationRead <- which(readTibble$Biotype == 'Mt_rRNA')
# readTibble <- readTibble[-mtRNALocationRead,]

In [101]:
# # Search for Genes of Interest
# umiTibble[which(umiTibble$'Gene Symbol' == 'ACTB'),]
# umiTibble[umiTibble$'Gene Symbol' == 'PGK1',]
# umiTibble[umiTibble$'Gene Symbol' == 'PPIA',]


_______